In [1]:
import os
import configparser

import pandas as pd

import psycopg2

import time

In [2]:
# 쿼리 조회 시간을 체크
def my_timer(original_function):

    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time() - t1
        print('{} 함수가 실행된 총 시간: {} 초'.format(original_function.__name__, t2))
        return result

    return wrapper

# @my_timer
# DB에서 데이터를 불러오기 위함
def query_select(query, params = []):

    root_path = os.path.dirname(os.getcwd())
    
    config  = configparser.ConfigParser()
    config.read(root_path + '/source/db_config.ini', encoding = 'utf-8')

    USERNAME = config['POSTGRES']['USERNAME']
    PASSWORD = config['POSTGRES']['PASSWORD']
    HOST = config['POSTGRES']['HOST']
    PORT = config['POSTGRES']['PORT']
    DATABASE = config['POSTGRES']['DATABASE']
    
    conn = psycopg2.connect(host=HOST,
                            user=USERNAME,
                            password=PASSWORD,
                            port=PORT,
                            database=DATABASE
                           )
    
    with conn.cursor() as cursor:
        
        if len(params) > 0:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
            
        headers = [x[0] for x in cursor.description]
        data = cursor.fetchall()
    
    conn.close()
       
    return pd.DataFrame(columns=headers, data=data)

In [3]:
# 환자에게 처방이 발생한 상병 중
# 상병 첫글자가 'a','b','c','d','e' 이면서
# 상병 중간에 heart를 포함를 포함 있는 상병
qry = """
    select 
        distinct b.concept_name
    from
        condition_occurrence a
        left join 
            (
                select *
                from concept 
                where 
                    substring(lower(concept_name),1,1) in ('a','b','c','d','e')
                    and lower(concept_name) like '%heart%'
            ) b
        on a.condition_concept_id = b.concept_id
    where concept_name is not null	
    ;
"""

In [4]:
result = query_select(qry)
result

,concept_name
0,Chronic congestive heart failure
